# Graph Experiment 2 bot

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from nnbench import NetMaker, NNMEG
import time
import tools.neotools as nj

In [ ]:
import neo4j
from collections import defaultdict

## Connecting

In [ ]:
driver = neo4j.GraphDatabase.driver("neo4j://neo4j:7687", auth=("neo4j", "test"))
driver.verify_connectivity()

# The graph database

In [ ]:
from IPython.display import SVG, display
display(SVG('ml graphdb structure r2.svg'))

# The experiment is set up in the graph
e.g. by [grex 2 prepare](grex%202%20prepare.ipynb)

## Methods to acquire work

In [ ]:
def get_experiment_key_from_name(driver, **kwargs):
    q="""
MATCH (e:Experiment {name: $name})
RETURN e.unikey as unikey
"""
    records = nj.query_read_return_list(driver, q, **kwargs)
    assert len(records) == 1
    return records[0]['unikey']

In [ ]:
def get_procedure_names_keys_from_experiment_key(driver, **kwargs):
    q = """
MATCH (:Experiment {unikey: $key})
-[:includes]->
(proc:Procedure)
RETURN proc.name as name, proc.unikey as key
"""
    return [(r['name'], r['key']) for r in nj.query_read_yield(driver, q, **kwargs)]

In [ ]:
def get_unstarted_parameters_of_procedure(driver, **kwargs):
    q="""
MATCH (:Procedure {unikey: $procedure_unikey})
-[:incorporates]->
(par:Parameters)
WHERE NOT (par)-[:configures]->(:net)
RETURN par.unikey as unikey
"""
    return [r['unikey'] for r in nj.query_read_yield(driver, q, **kwargs)]

In [ ]:
def get_code_strings_of_experiment_procedure_parameters(driver, **kwargs):
    q = """
MATCH (e:Experiment {unikey: $experiment_unikey})
-[:includes]->
(proc:Procedure {unikey: $procedure_unikey})
-[:incorporates]->
(par:Parameters {unikey: $parameters_unikey})
RETURN proc.code_strings, par.code_strings
"""
    records = nj.query_read_return_list(driver, q, **kwargs)
    assert len(records) == 1
    r = records[0]
    #print(f"r['proc.code_strings'] = {r['proc.code_strings']}")
    #print(f"r['par.code_strings'] = {r['par.code_strings']}")
    return r['proc.code_strings'] + r['par.code_strings']

## Find work and do it

In [ ]:
def get_code_strings_from_db(driver, experiment_name, procedure_name):
    experiment_unikey = get_experiment_key_from_name(driver, name=experiment_name)
    #print(experiment_unikey)
    procedures = dict(get_procedure_names_keys_from_experiment_key(driver, key=experiment_unikey))
    procedure_unikey = procedures[procedure_name]
    #print(procedure_unikey)
    unstarted_parameters = get_unstarted_parameters_of_procedure(driver, procedure_unikey=procedure_unikey)
    parameters_unikey = unstarted_parameters[0]
    #print(parameters_unikey)
    code_strings_from_db = get_code_strings_of_experiment_procedure_parameters(driver,
        experiment_unikey=experiment_unikey,
        procedure_unikey=procedure_unikey,
        parameters_unikey=parameters_unikey)
    return code_strings_from_db

In [ ]:
def now_run_it(driver, code_strings):
    cx = {}
    for s in code_strings:
        exec(s, cx)
    nps = nj.NumpyStore(driver)
    cx['run_it'](cx, driver, nps)

In [ ]:
code_strings = get_code_strings_from_db(driver, 't2', 'Train ADCs')

In [ ]:
now_run_it(driver, code_strings)

___

# Stop

In [ ]:
assert False, "stop here"